## Домашнее задание 6, Ускорение и оптимальные методы

### Deadline - 25.10.2024    23:59

# Подготовительная часть

как и ранее, рассмотрит задачу минимизации эмпирического риска:

\begin{equation}
\min_{x \in \mathbb{R}^d} \left[ f(x) = \frac{1}{n} \sum\limits_{i=1}^n \ell_x(a_i, b_i) + \frac{\lambda}{2} \| x \|^2_2\right],
\end{equation}

где:
- $\ell_x(a_i, b_i)$ — функция потерь (cross-entropy loss),
- $x$ — вектор параметров модели,
- $\{a_i, b_i\}_{i=1}^n$ — выборка данных,
- $\lambda > 0$ — параметр регуляризации.

Функция потерь для каждого объекта $i$ записывается как:

\begin{equation}
\ell_x(a_i, b_i) = -b_i \ln(p(x^Ta_i)) - (1 - b_i) \ln(1 - p(x^Ta_i)),
\end{equation}

где $p(x^Ta_i)$ — это вероятность, вычисляемая с помощью логистической функции в комбинации с линейной моделью:

\begin{equation}
p(x^Ta_i) = \frac{1}{1 + \exp(-x^T a_i)}.
\end{equation}




К заданию приложен датасет _mushrooms_. С помощью следующего кода сформируйте матрицу $A$ и вектор $b$, в которой и будет храниться выборка $\{a_i, b_i\}_{i=1}^n$. 

In [ ]:
dataset = "mushrooms.txt" 
#файл должен лежать в той же деректории, что и notebook

In [ ]:
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
A, b = data[0].toarray(), data[1]

Поменяем вектор $y$, чтобы $y_i$ принимали значения $0$ и $1$. Вы также можете сделать дополнительную предобработку данных (приемами из машинного обучения), но это никак дополнительно не оценивается.

In [ ]:
b = b - 1

Разделим данные на две части: обучающую и тестовую.

In [ ]:
from sklearn.model_selection import train_test_split
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=GLOBAL_SEED)

Для обучающей части $A_{train}$, $b_{train}$ оцените константу $L$. Задайте $\lambda$ так, чтобы $\lambda \approx L / 1000$.  Реализуйте в коде подсчет значения градиента для нашей целевой функции ($A$, $b$, $\lambda$ лучше подавать в качестве параметра, чтобы была возможность их менять, а не только подставлять фиксированные $A_{train}$, $b_{train}$). Можно использовать как библиотеку ``numpy``, так и библиотеки ``autograd``, ``pytorch``, ``jax``. Воспользуйтесь кодом с предыдущего домашнего задания.

## Основная часть (всего 5 баллов) 

__Задача 1. (всего 2.5 балла)__ Метод Ньютона (классический и демпфированный)

Рассмотрим один из самых известных методов второго порядка - Метод Ньютона (Newton's method). А точнее его модификацию - демпфированный метод Ньютона (Dumped Newton's method, DNM). 

**Псевдокод алгоритма**

_Инициализация:_

Величина шага $\{ \gamma_k \}_{k=0} > 0$, стартовая точка $ x^0 \in \mathbb{R}^d $, количество итераций $ K $

$k \hspace{-1em}$ _--ая итерация:_
1. Подсчитать направление спуска $$ d_k = \left( \nabla^2 f(x^k) \right)^{-1} \nabla f(x_k)  $$
2. Сделать шаг алгоритма $$ x^{k+1} = x^k - \gamma_k d_k $$

Используйте предложенную функцию для реализации алгоритма и допишите недостающие фрагменты. После чего для проверки правильности загрузите функцию в [контест](https://contest.yandex.ru/contest/66540/enter/)

Как нетрудно заметить, при $\gamma_k \equiv 1$ демпфированный метод вырождается в классический (который, наверное, вам известен из вычислительной математики). 

### Указание: поставьте во всех методах точность по умолчанию равной $10^{-16}$.

__а) (0.25 балла)__ Для метода Ньютона нам нужно будет использовать значение гессиана функции потерь. Воспользуясь знаниями, полученными ранее, докажите, что градиент и гессиан равны, соответственно
$$
\nabla f(x) = \frac{1}{n} \sum_{i=1}^n (p(x^Ta_i) - b_i) a_i + \lambda x \qquad \qquad \nabla^2 f(x) = \frac{1}{n} \sum_{i=1}^n \big(p(x^Ta_i)(1 - p(x^Ta_i))\big) a_i a_i^T + \lambda I_d
$$

__Ваше решение__

__б) (0.25 балла)__ Запустите демпфированный метод Ньютона для задачи логистической регрессии. Пусть $x_0 = 0$. Постройте график зависимости критерия от числа итераций (```max_iter``` поставьте не больше 10). Сходится ли метод?

In [ ]:
import numpy as np
from tqdm import trange

def DumpedNewton(grad, hess, criterion, x_0, eps, max_iter, **params):

    '''
       grad(x) - функция, которая считает градиент целевой функции;
       hess(x) - функция, которая считает гессиан целевой функции;
       criterion(x) - функция, которая считает критерий;
       x_0 - начальная точка;
       eps - точность сходимости (обычно 1e-8);
       max_iter - количество итераций;
       **params - содержит именнованные гиперпараметры метода:
           params['gamma'] - шаг;
           params['flag] -- флаг продолжения вычислений.
    '''

    errors = []

    x_k = np.copy(x_0)
    err_x_0 = criterion(x_k)

    if params['flag']:
        errors.append(criterion(x_k))
    else:
        errors.append(criterion(x_k) / err_x_0)

    for k in trange(max_iter):
            
        # Ваше решение

        if errors[-1] < eps:
            break
            
    return x_k, errors

In [ ]:
# Ваше решение (Code + Markdown)

__в) (0.25 балла)__ Зафиксировав значение $\gamma_k=1$, рассмотрите следующие значения парметра $x_0$:
1. Вектор из 0;
2. Вектор из 1;
3. Вектора $\vec{e}_i$ ($i = 1, 2, \ldots , 8$), $j$-ая координата которых подчиняется следующему правилу:
$$  \left[ \vec{e}_i \right]_j = \begin{cases} i, & \text{если } j \leq i \\ 0, &\text{иначе} \end{cases}$$

Постройте графики зависимости критерия от числа итераций.

In [ ]:
# Ваше решение (Code)

__г) (0.5 балла)__ Выберите и зафиксируйте произвольное значение $x_0$ из предыдущего пункта, для которого наблюдается сходимость DNM. Теперь исследуем зависимость сходимости от параметра $\gamma_k$. Для этого проверьте следующие значения $\gamma_k = \{ 0.5, 1, 1.5, 2 \}$. Постройте графики зависимости критерия от числа итераций.

In [ ]:
# Ваше решение (Code)

__д) (0.75 балла)__ Давайте теперь вызовем несколько итераций метода градиентного спуска, перед использованием демпфированного метода Ньютона (воспользуйтесь результатами предыдущих домашек). Варьируя число итераций градиентного спуска (возьмите 10, 20, 50, 100), исследуйте сходимость метода DNM при фиксированных значениях $x_0$ (возьмите вектор из единиц) и $\gamma_k = 1$. 

In [3]:
# Ваше решение (Code)

__е) (0.25 балла)__ Теперь рассмотрим датасет ```w8a.txt```. Его размеры несколько больше, чем у ```mushrooms.txt``` (сравните, насколько), поэтому вычисление гессиана будет занимать больше времени. Повторите предыдущий пункт, используя те же значения для количества предварительных шагов градиентного спуска, а число итераций демпфированного метода Ньютона положите равной 5.

In [ ]:
dataset = "w8a.txt" 

from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

data = load_svmlight_file(dataset, )
A, b = data[0].toarray(), data[1]

A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=GLOBAL_SEED)

In [ ]:
# Ваше решение (Code)

__ё) (0.25 балла)__ В данном пункте нужно будет сравнить временную сложность алгоритмов первого порядка (```GradientDescent, HeavyBall, NAG```) и предложенных в данной домашней работе реализаций метода Ньютона (```DumpedNewton``` и связку ```GradientDescent + DumpedNewton```). Для всех методов выберите $x_0 = 0$, $\varepsilon = 10^{-16}$ и следующие значения параметров:
1. ```GradientDescent```: max_iter = 3000, $\gamma_k = 1/L$;
2. ```HeavyBall```: max_iter = 3000, $\gamma_k = 1/L$, $\tau_k = 0.9$;
3. ```NAG```: max_iter = 3000, $\gamma_k = 1/L$, $\tau_k = 0.9$;
4. ```DumpedNewton```: max_iter = 8, $\gamma_k = 1$;
5. ```GradientDescent + DumpedNewton```: число шагов градиентного спуска - 100, число шагов метода Ньютона - 5, параметры методов такие же, как в пунктах 1 и 4.

Сделайте вывод о применимости метода Ньютона.


In [4]:
# Ваше решение (Code + Markdown)

__Задача 2. (всего 2.5 балла)__ Квазиньютоновские методы

Вычисление обратной матрицы Гесса является крайне ресурсоемким, поэтому предлагается использовать вместо ее точного значения некоторую аппроксимацию невырожденной положительно определенной матрицей. Такие методы называются *квазиньютоновскими*. В данной части будет предложен к рассмотрению один из самых популярных методов оптимизации.

__а) (0.5 балла)__ Реализуйте квазиньютоновский метод L-BFGS (см. [статью](https://users.iems.northwestern.edu/~nocedal/PDFfiles/limited-memory.pdf)), основанный на аппроксимации $H_k$. 

**Псевдокод алгоритма**

_Инициализация:_

Параметры линейного поиска $c_1, c_2$, аппроксимация обратного гессиана $H_0$, стартовая точка $ x^0 \in \mathbb{R}^d $, количество итераций $ K $

$k \hspace{-1em}$ _--ая итерация:_

1. Подсчитать направление спуска, используя аппроксимацию обратного гессиана $H_k$:
$$d_k = -H_k \nabla f(x_k).$$
2. Выполнить процедуру линейного поиска параметра $\alpha_k$, удовлетворяющего условиям Вольфе (см. [определение](https://en.wikipedia.org/wiki/Wolfe_conditions)):
$$\begin{align*}
f(x_k + \alpha_k d_k) &\leq f(x_k) + c_1 \alpha_k d_k^T \nabla f(x_k)\\
-d_k^T \nabla f(x_k + \alpha_k d_k)  &\leq -c_2 d_k^T \nabla f(x_k)
\end{align*}$$ 
3. Обновить $x_k$:
$$x_{k + 1} = x_k + \alpha_k d_k$$
4. Ввести $$\begin{align*} s_k &= x_{k + 1} - x_k \\ y_k &= \nabla f(x_{k + 1}) - \nabla f(x_k)\end{align*}$$ и обновить значение $H_k$:
$$
H_{k +1} = \left( I - \frac{y_k s_k^T}{y_k^T s_k} \right)^T H_k \left( I - \frac{y_k s_k^T}{y_k^T s_k} \right) + \frac{y_k s_k^T}{y_k^T s_k}
$$

*Замечание*: подумайте, как лучше инициализировать матрицу $H_0$. Функцию линейного поиска можно реализовать самостоятельно, а можете воспользоваться реализацией из библиотеки ```scipy.optimize```. 

Используйте предложенную функцию для реализации алгоритма и допишите недостающие фрагменты. После чего для проверки правильности загрузите функцию в [контест](https://contest.yandex.ru/contest/66540/enter/)

In [ ]:
from tqdm import trange
from scipy.optimize import line_search

def BFGS(f, grad, H_0, x_0, criterion, eps, max_iter, **params):
    '''
        f(x) - оптимизируемая функция
        grad(x) - функция, которая считает градиент целевой функции;
        H_0 - аппроксимация гессиана невырожденной положительно определенной матрицей;
        criterion(x) - функция, которая считает критерий;
        x_0 - начальная точка;
        eps - точность сходимости (обычно 1e-8);
        max_iter - количество итераций;
        **params - содержит именнованные гиперпараметры метода:
            params['c_1'] - параметр в условие Вольфе, по умолчанию равен 0.0001
            params['c_2'] - параметр в условие Вольфе, по умолчанию равен 0.9
    '''
    errors = []

    err_x_0 = criterion(x_0)
    errors.append(criterion(x_0) / err_x_0)

    H_k = np.copy(H_0)

    x_k = np.array(x_0)
    x_new = np.array(x_0)

    g_k = grad(x_0)
    g_new = grad(x_0)

    I = np.eye(d)

    for k in trange(max_iter):

        # Ваше решение
        
        errors.append(criterion(x_new) / err_x_0)
        if errors[-1] < eps:
            break

    return x_new, errors

Вернемся к датасету ```mushrooms.txt```, для ускорения работы алгоритмов.

In [ ]:
dataset = "mushrooms.txt" 

from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

data = load_svmlight_file(dataset)
A, b = data[0].toarray(), data[1]

b = 2 * b - 3

A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=GLOBAL_SEED)

__б) (0.5 балла)__ Рассмотрите различные значения параметра $x_0$, как в пункте __1б)__. Число итераций поставьте равным 150. Постройте график зависимости критерия от номера итерации. Проведите сравнение с подобными графиками для DNM.

In [ ]:
# Ваше решение (Code)

__в) (0.75 балла)__ Однако, такое приближение гессианов не единственно. Реализуйте метод SR1 (Symmetric rank-1) (см. [статью](https://arxiv.org/pdf/2002.00657.pdf)) со следующим обновлением аппроксимации обратного гессиана:
$$
H_{k + 1} = H_k + \frac{(s_k - H_k y_k)(s_k - H_k y_k)^T}{(s_k - H_k y_k)y_k^T}.
$$

In [ ]:
from tqdm import trange
from scipy.optimize import line_search

def SR1(f, grad, H_0, x_0, criterion, eps, max_iter, **params):
    '''
        f(x) - оптимизируемая функция
        grad(x) - функция, которая считает градиент целевой функции;
        H_0 - аппроксимация гессиана невырожденной положительно определенной матрицей;
        criterion(x) - функция, которая считает критерий;
        x_0 - начальная точка;
        eps - точность сходимости (обычно 1e-8);
        max_iter - количество итераций;
        **params - содержит именнованные гиперпараметры метода:
            params['c_1'] - параметр в условие Вольфе, по умолчанию равен 0.0001
            params['c_2'] - параметр в условие Вольфе, по умолчанию равен 0.9
    '''
    errors = []

    err_x_0 = criterion(x_0)
    errors.append(criterion(x_0) / err_x_0)

    H_k = np.copy(H_0)

    x_k = np.array(x_0)
    x_new = np.array(x_0)

    g_k = grad(x_0)
    g_new = grad(x_0)

    for k in trange(max_iter):

        # Ваше решение

        errors.append(criterion(x_new) / err_x_0)
        if errors[-1] < eps:
            break

    return x_new, errors

__г) (0.75 балла)__ Теперь давайте сравним работу квазиньютоновских методов с ускоренным методом (возьмите NAG) и методом второго порядка (DumpedNewton). В качестве начальной точки выберите $x_0 = 0$. Постройте сравнительные графики зависимостей критерия от итерации и критерия от времени. Что можно сказать о применимости квазиньютоновских методов? 

In [ ]:
# Ваше решение (Code + Markdown)

## Дополнительная часть (всего 5 баллов)

__Задача 3. (всего 5 баллов)__ Квазиньютоновские методы очень хороши, так как не требуют использование второго порядка гладкости. Однако, все равно приходится хранить приближения матрицы Гесса, что при больших размерностях задачи очень сильно влияет на асимптотику по памяти, делая ее квадратичной. Для оптимизации этой величины были представлены методы с представкой L- (L-BFGS, L-SR1), где L- означает limited memory.

__a) (1 балл)__ Реализуйте метод L-BFGS, использующий технику ограниченной памяти с асимптотикой $\mathcal{O}(n)$. Пусть разрешено использовать $m$ значений с предыдущих итераций. Тогда псевдокод на $k$-ой итерации:

**Псевдокод алгоритма**

_Инициализация:_

Параметры линейного поиска $c_1, c_2$, стартовая аппроксимация обратного гессиана $H_0^k$, стартовая точка $ x^0 \in \mathbb{R}^d $, количество итераций $ K $

$k \hspace{-1em}$ _--ая итерация:_

1. Создать переменную $q = g_k$.
2. Для $i = k - 1, \ldots, k - m$ уточнить аппроксимацию градиента $q$, сохранив при этом вычисленное значение $\alpha_i$:
$$
\alpha_i =\rho_i s_i^T q \qquad \qquad q = q - \alpha_i y_i.
$$
3. Задать нулевое приблежение обратного гессиана $H_k^0$ (как в статье, вы же можете рассмотреть другое):
$$
H_k^0 = \frac{s_{k-1}^T y_{k-1}}{y_{k-1}^T y_{k-1}} I.
$$
4. Вычислить значение спуска:
$$
r = H_k^0 q
$$
5. Для $i = k - m, \ldots, k - 1$ уточнить значение вектора спуска $r$:
$$
r = r + s_i (\alpha_i - \rho_i y_i^T r).
$$
6. Сделать стандартные обновления для подсчета $x_{k + 1}, s_k, y_k, \rho_k$, сохранить их в память. Если память переполнена -- убрать элемент с индексом 0 (то есть тот, который был положен раньше остальных).

Используйте предложенную функцию для реализации алгоритма и допишите недостающие фрагменты. После чего для проверки правильности загрузите функцию в [контест](https://contest.yandex.ru/contest/66540/enter/)

In [ ]:
import copy
def LBFGS(f, grad, H_0, x_0, mem_size, criterion, eps, max_iter, **params):
    '''
        f(x) - оптимизируемая функция
        grad(x) - функция, которая считает градиент целевой функции;
        H_0 - аппроксимация гессиана невырожденной положительно определенной матрицей;
        mem_size - количество выделенной памяти, по умолчанию 1;
        criterion(x) - функция, которая считает критерий;
        x_0 - начальная точка;
        eps - точность сходимости (обычно 1e-8);
        max_iter - количество итераций;
        **params - содержит именнованные гиперпараметры метода:
            params['c_1'] - параметр в условие Вольфе, по умолчанию равен 0.0001
            params['c_2'] - параметр в условие Вольфе, по умолчанию равен 0.9
    '''
    errors = []
    time_logs = []

    err_x_0 = criterion(x_0)
    errors.append(criterion(x_0) / err_x_0)

    H_k = np.copy(H_0)

    x_k = np.array(x_0)
    x_new = np.array(x_0)

    g_k = grad(x_0)
    g_new = grad(x_0)

    I = np.eye(d)

    time_logs.append(0)

    x_diffs = []
    grad_diffs = []
    rhos = []

    for k in trange(max_iter):

        # Ваше решение

        if errors[-1] < eps:
            break

    return x_new, errors, time_logs

__б) (0.5 балла)__ Сравните сходимость метода при разных значениях параметра ```mem_size```. Рассмотрите следующие значения: 1, 10, 100, 1000, 10000. Постройте графики зависимости критерия от номера итерации и критерия от времени. Сравните также с BFGS. Что можно сказать об алгоритме с ограниченной памятью?

In [ ]:
# Ваше решение (Code)

__в)__ __(2 балла)__ До этого мы рассматривали методы, требующие липшицевость градиента. Однако, существует алгоритм, требующий липшицевость гессиана. Это метод Ньютона с кубической регуляризацией, предложенный Ю.$~$Нестеровым и Б.$~$Т.$~$Поляком в [этой работе](https://link.springer.com/article/10.1007/s10107-006-0706-8). На его $k$-ой итерации необходимо отрешивать следующую подзадачу:
$$
x_{k + 1} = \arg \min \limits_{x \in \mathbb{R}^d} \left\{ \langle \nabla f(x_k), x - x_k \rangle + \frac{1}{2} \langle \nabla^2 f(x_k) (x - x_k), x - x_k \rangle + \frac{M_k}{6} \| x - x_k\|_2^2 \right\}
$$
Здесь $M_k$ -- константа Липшица гессиана. Так как в общем случае вычислять тензор третьего ранга не очень выгодно, предлагается рассмотреть адаптивную константу, которую мы будем задавать как
$$
M_k = \frac{\alpha}{\left( 1 + \beta^{1 + k} \right)},
$$
константы $\alpha$ и $\beta$ для начала можно положить равными 0.005 и 0.15, соответственно.

In [ ]:
def solve_cubic(x_k, f, g_k, H_k, M, criterion, eps=1e-8, max_iter=10):
    '''
        x_k - точка, в которой решается задача минимизации;
        f(x) - целевая функция:
        g_k - значение градиента функции в точке x_k;
        H_k - значение гессиана функции в точке x_k;
        M - константа Липшица гессиана;
        criterion(x, y) - функция, считающая критерий останова, в оригинальной статье взят 1/||x|| - 1/||y||;
        eps - точность сходимости (обычно 1e-8);
        max_iter - количество итераций для решения подзадачи, по умолчанию 10;
    '''
    # Inner variable to trace amount of steps made inside the subproblem
    solver_it = 1

    newton_step = -np.linalg.solve(H_k, g_k)
    if M == 0:
        return x_k + newton_step, solver_it

    def cauchy_point(g, H, M):
        if np.linalg.norm(g) == 0 or M == 0:
            return 0 * g

        g_dir = g / np.linalg.norm(g)
        H_g_g = H @ g_dir @ g_dir
        R = -H_g_g / (2*M) + np.sqrt((H_g_g/M)**2 / 4 + np.linalg.norm(g)/M)
        return -R * g_dir

    r_min = np.linalg.norm(cauchy_point(g_k, H_k, M))

    x_new = x_k + newton_step
    if f(x_k) > f(x_new):
        return x_new, solver_it

    r_max = np.linalg.norm(newton_step)
    if r_max - r_min < eps:
        return x_k + newton_step, solver_it

    I = np.eye(len(g_k))

    for _ in range(max_iter):
        r_try = (r_min + r_max) / 2
        lam = r_try * M
        s_lam = -np.linalg.solve(H_k + lam * I, g_k)
        solver_it += 1

        crit = 1./np.linalg.norm(s_lam) - 1./np.linalg.norm(r_try)
        if np.abs(crit) < eps:
            return x_k + s_lam, solver_it

        if crit < 0:
            r_min = r_try
        else:
            r_max = r_try
        if r_max - r_min < eps:
            break
    return x_k + s_lam, solver_it

In [ ]:
from tqdm import tqdm

def CubicNewton(f, grad, H, x_0, criterion, eps, max_iter, **params):
    '''
        f(x) - оптимизируемая функция
        grad(x) - функция, которая считает градиент целевой функции;
        H(x) - функция, которая считает гессиан целевой функции;
        criterion(x) - функция, которая считает критерий;
        x_0 - начальная точка;
        eps - точность сходимости (обычно 1e-8);
        max_iter - количество итераций;
        **params - содержит именнованные гиперпараметры метода:
            params['M'] - константа Липшица гессиана, она же константа кубического регуляризатора
    '''

    errors = []

    err_x_0 = criterion(x_0)
    errors.append(criterion(x_0) / err_x_0)

    x_k = np.array(x_0)
    g_k = grad(x_0)
    H_k = H(x_0)

    for k in tqdm(range(max_iter)):

        M_k = params['M'](k)

        x_k, solver_it = solve_cubic(x_k, f, g_k, H_k, M_k, criterion, eps=1e-16, max_iter=2)
        k += solver_it
        g_k = grad(x_k)
        H_k = H(x_k)

        errors.append(criterion(x_k) / err_x_0)
        if errors[-1] < eps:
            break

    return x_k, errors

__г) (0.25 балла)__ Примените кубически регуляризованный метод Ньютона для ERM. Постройте график зависимости значения критерия от итерации.

In [ ]:
# Решение (Code)

__д) (0.25 балла)__ Давайте подберем теперь наилучшие значения параметров $\alpha, \beta$, обеспечивающих наискорейшую сходимость метода. Для этого рассмотрите $\alpha = \{ 10^{-1}, 10^{-2}, 10^{-3}, 10^{-4} \}$ и $\beta = \{ 1, 0.5, 0.1, 0.05 \}$.

In [ ]:
# Ваше решение (Code)

__е) (1 балл)__ И финальное сравнение - постройте сравнительные графики для всех изученных методов в данной домашней работе. Параметры подберите так, чтобы они обеспечивали наилучшую сходимость методов. 

In [ ]:
# Ваше решение (Code + Markdown)